<img src="./pictures/logo_sizinglab.png" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Sizing of a multi-rotor drone

*Written by Marc Budinger, Aitor Ochotorena (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

The objective of this notebook is to select the best compromise of components (propeller, motor, ESC, battery) of a multi-rotor drone for given specifiations.

**Scipy** and **math** packages will be used for this notebook for the optimization algorithms of python.

**widgets** provides an interactive layout to display the results.

In [3]:
import scipy
import scipy.optimize

from math import pi
from math import sqrt
from math import sin

import numpy as np
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display

pd.options.display.float_format = '{:,.2f}'.format

## Contents
1. [Sizing the propeller for vertical climb speed ](#section_1)
2. [Architecture defintion and design assumptions](#section_2)
3. [Sizing code](#section_3)
4. [Optimization problem](#section_4)

<a id='section_1'></a>

## Sizing propeller to vertical climb speed scenario

This exercise targets to minimize the propeller torque for vertical climb speed scenario. This Notebook shows the role of the design variables as well as the constraints in an optimization problem. 


Let us say that $Q_{pro,cl}$ is the climb torque to minimize. 

For the vertical climb, advance ratio ($J=\frac{V_{cl}}{n_{cl}   D_{pro}}$) is a key parameter to estimate the vertical speed. 

Given that $D_{pro}$ and $V_{cl}$ are known from the specifications of the problem, we face a problem with one equation and two undetermined parameters.

**STRATEGY:** One of these parameters is going to be defined within a design range (e.g. J) and thus, the rest of calculations are done. This particular case is called **underconstraint singularity**.

Finally, estimated parameter is compared to the real value through the constraint. 

The sizing code is defined here in a function which can give:
- an evaluation of the objective: here minimize the torque in climb scenario
- an evaluation of the constraints: 



<math>\begin{align}
&\underset{\mathbf{x}}{\operatorname{minimize}}& & Q_{pro,cl}(\mathbf{x}) \\
&\operatorname{subject\ to}
& & 5\% \leq  V_{cl}-J n_{cl}   D_{pro} \leq 0 \\
\end{align}</math>

where it is expected that the product $J n_{cl}   D_{pro}$ calculated does not differ more than a 5\%.

Here is an non-exhaustive XDSM diagram of the whole multirotor sizing code:

![XDSM](pictures/xdsm_multirotor_mdo.png)

More details in the setting up of sizing code can be found in the  [following paper](https://www.researchgate.net/profile/Marc_Budinger/publication/277933677_Computer-aided_definition_of_sizing_procedures_and_optimization_problems_of_mechatronic_systems/links/55969de508ae793d137c7ea5/Computer-aided-definition-of-sizing-procedures-and-optimization-problems-of-mechatronic-systems.pdf):  

> Reysset, A., Budinger, M., & Maré, J. C. (2015). Computer-aided definition of sizing procedures and optimization problems of mechatronic systems. Concurrent Engineering, 23(4), 320-332.


<a id='section_2'></a>

## Architecture definition and design assumptions

In [4]:
M_total = 1.35 # [kg] mass of the drone 

rho_air=1.18# [kg/m^3] Air density

pitch=4.5*0.0254#[m] propeller pitch

a_to = 0.25 * 9.81 # [m/s²] acceleration during take off

N_arm = 4 # [-] number of arm

N_pro_arm = 1 # [-] number of propeller per arm (1 or 2)

N_pro = N_pro_arm * N_arm # [-] Propellers number

ND_max=105000./60.*.0254 #[Hz.m] Max speed limit (N.D max) for APC MR propellers

# Reference parameters for scaling laws
D_pro_ref=11.*.0254# [m] Reference propeller diameter
M_pro_ref=0.53*0.0283# [kg] Reference propeller mass

<a id='section_3'></a>

## Sizing code

In [9]:
# -----------------------
# sizing code
# -----------------------
# inputs: 
# - param: optimisation variables vector (reduction ratio, oversizing coefficient)
# - arg: selection of output  
# output: 
# - objective if arg='Obj', problem characteristics if arg='Prt', constraints other else

def sizing_code(param, arg):

# Design variables
# ---
    beta_pro = param[0] # pitch/diameter ratio of the propeller
    k_ND = param[1] # slow down propeller coef : ND = kNDmax / k_ND

    F_pro_hov = M_total * (9.81) / N_pro # [N] Thrust per propeller for hover
    F_pro_to = M_total * (9.81 + a_to) / N_pro # [N] Thrust per propeller for take-off
    
    C_t = 4.27e-02 + 1.44e-01 * beta_pro  # Thrust coef with T=C_T.rho.n^2.D^4 - 0.8 for de-rating of APC catalog
    C_p = -1.48e-03 + 9.72e-02 * beta_pro  # Power coef with P=C_p.rho.n^3.D^5

    # Propeller selection with take-off scenario
    D_pro = (F_pro_to / (C_t*rho_air*(ND_max/k_ND)**2.))**0.5 # [m] Propeller diameter
    n_pro_to = ND_max / k_ND / D_pro # [Hz] Propeller speed 
    Omega_pro_to = n_pro_to * 2*pi # [rad/s] Propeller speed

    M_pro = M_pro_ref * (D_pro/D_pro_ref)**2. # [kg] Propeller mass

    P_pro_to = C_p * rho_air * n_pro_to**3. * D_pro**5. # [W] Power per propeller
    T_pro_to = P_pro_to / Omega_pro_to # [N.m] Propeller torque

    # Propeller torque & speed for hover
    n_pro_hov = sqrt(F_pro_hov/(C_t * rho_air *D_pro**4.)) # [Hz] hover speed
    Omega_pro_hov = n_pro_hov * 2.*pi # [rad/s] Propeller speed

    P_pro_hov = C_p * rho_air * n_pro_hov**3. * D_pro**5. # [W] Power per propeller
    T_pro_hov = P_pro_hov / Omega_pro_hov # [N.m] Propeller torque       

# Objective and Constraints sum up
# ---
    obj =  10 *  M_pro # minimize propeller mass

    constraints = []

    # Run algorithm slsqp
    if arg == 'Obj':
        return obj # for torque optimisation

     # Run algorithm differential evolution
    elif arg == 'ObjP':
        P = 0. # Penalisation nulle
        for C in constraints: 
            if (C < 0.): 
                P = P-1e9*C
        return obj + P # for torque optimisation       

    elif arg=='Prt':

        col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']

        df = pd.DataFrame()
        df = df.append([{'Type': 'Optimization', 'Name': 'beta_pro', 'Value': beta_pro, 'Unit': '[-]', 'Comment': 'pitch/diameter ratio'}])[col_names]
        df = df.append([{'Type': 'Optimization', 'Name': 'k_ND', 'Value': k_ND, 'Unit': '[-]', 'Comment': 'slow down propeller coef'}])[col_names]
        df = df.append([{'Type': 'Optimization', 'Name': 'Objective',  'Value': obj, 'Unit': '[kg]', 'Comment': 'Propeller mass'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'beta_pro', 'Value': beta_pro, 'Unit': '[-]', 'Comment': 'Ratio pitch to diameter'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'F_pro_to', 'Value': F_pro_to, 'Unit': '[N]', 'Comment': 'Thrust for 1 propeller during Take Off'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'F_pro_hov', 'Value': F_pro_hov, 'Unit': '[N]', 'Comment': 'Thrust for 1 propeller during Hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'rho_air', 'Value': rho_air, 'Unit': '[kg/m^3]', 'Comment': 'Air density'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'Dpro_ref', 'Value': D_pro_ref, 'Unit': '[m]', 'Comment': 'Reference propeller diameter'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'M_pro_ref', 'Value': M_pro_ref, 'Unit': '[kg]', 'Comment': 'Reference propeller mass'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'C_t', 'Value': C_t, 'Unit': '[-]', 'Comment': 'Thrust coefficient of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'C_p', 'Value': C_p, 'Unit': '[-]', 'Comment': 'Power coefficient of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'D_pro', 'Value': D_pro, 'Unit': '[m]', 'Comment': 'Diameter of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'n_pro_to', 'Value': n_pro_to, 'Unit': '[Hz]', 'Comment': 'Rev speed of the propeller during takeoff'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'n_pro_hov', 'Value': n_pro_hov, 'Unit': '[Hz]', 'Comment': 'Rev speed of the propeller during hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'P_pro_to', 'Value': P_pro_to, 'Unit': '[W]', 'Comment': 'Power on the mechanical shaft of the propeller during takeoff'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'P_pro_hov', 'Value': P_pro_hov, 'Unit': '[W]', 'Comment': 'Power on the mechanical shaft of the propeller during hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'M_pro', 'Value': M_pro, 'Unit': '[kg]', 'Comment': 'Mass of the propeller'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'Omega_pro_to', 'Value': Omega_pro_to, 'Unit': '[rad/s]', 'Comment': 'Rev speed of the propeller during takeoff'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'Omega_pro_hov', 'Value': Omega_pro_hov, 'Unit': '[rad/s]', 'Comment': 'Rev speed of the propeller during hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'T_pro_hov', 'Value': T_pro_hov, 'Unit': '[N.m]', 'Comment': 'Torque on the mechanical shaft of the propeller during hover'}])[col_names]
        df = df.append([{'Type': 'Propeller', 'Name': 'T_pro_to', 'Value': T_pro_to, 'Unit': '[N.m]', 'Comment': 'Torque on the mechanical shaft of the propeller during takeoff'}])[col_names]
    

        items = sorted(df['Type'].unique().tolist())
        def f(Type):
            return df[df['Type']==Type] 
        widgets.interact(f, Type=items)
        return f
             
    else:
        return constraints


<a id='section_4'></a>

## Optimization problem


We will now use the [optimization algorithms](https://docs.scipy.org/doc/scipy/reference/optimize.html) of the Scipy package to solve and optimize the configuration. We can use the SLSQP algorithm without explicit expression of the gradient (Jacobian) or a differential evolution optimization to have a more exhaustive calculation. A course on Multidisplinary Gradient optimization algorithms and gradient optimization algorithm is given [here](http://mdolab.engin.umich.edu/sites/default/files/Martins-MDO-course-notes.pdf):
> Joaquim R. R. A. Martins (2012). A Short Course on Multidisciplinary Design Optimization. University of Michigan


The first step is to give an initial value of optimisation variables:

In [10]:
# Optimisation variables
beta_pro = .33 # pitch/diameter ratio of the propeller
k_ND = 1.2 # slow down propeller coef : ND = kNDmax / k_ND

# Vector of parameters
parameters = scipy.array((beta_pro, k_ND))

We can print of the characterisitcs of the problem before optimization with the initial vector of optimization variables. Through the droplist menu, an interactive display of the parameters grouped by their type of variable is given:

In [11]:
# Initial characteristics before optimization 
print("-----------------------------------------------")
print("Initial characteristics before optimization :")
sizing_code(parameters,'Prt')
print("-----------------------------------------------")

-----------------------------------------------
Initial characteristics before optimization :


interactive(children=(Dropdown(description='Type', options=('Optimization', 'Propeller'), value='Optimization'…

-----------------------------------------------


Then we can solve the problem and print of the optimized solution:

In [12]:
# Optimization with SLSQP algorithm
contrainte = lambda x: sizing_code(x, 'Const')
objectif = lambda x: sizing_code(x, 'Obj')
objectifP = lambda x: sizing_code(x, 'ObjP')

SLSQP = False # Optimization algorithm choice


# Optimization bounds
# beta, k_ND
bounds = [(0.3,0.6), (1,100)]

if SLSQP == True:
    # SLSQP omptimisation
    result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, 
                                   bounds=bounds,
                                   f_ieqcons=contrainte, iter=1500, acc=1e-12)
else:
    # Differential evolution omptimisation
    result = scipy.optimize.differential_evolution(func=objectifP,
                                   bounds=bounds,
                                   tol=1e-12)

# Final characteristics after optimization 
print("-----------------------------------------------")
print("Final characteristics after optimization :")

if SLSQP == True:
    sizing_code(result,'Obj')
    sizing_code(result, 'Prt')
else:
    sizing_code(result.x,'Obj')
    sizing_code(result.x, 'Prt')
print("-----------------------------------------------")



-----------------------------------------------
Final characteristics after optimization :


interactive(children=(Dropdown(description='Type', options=('Optimization', 'Propeller'), value='Optimization'…

-----------------------------------------------
